# Summary
Needs to be cleaned up!

Generates a MapInfo File with hexagons and their stats.

Uses the custom geometry which splits the hexagons along city borders
and subsequently merges small and medium (PCCLASS 2,3) cities into single 
areas. (Which is a file output from elsewhere, technically could be computed
here as well). 

It combines the above against tiles from the Ookla dataset. This 
is done by calculating the spatial join (GeoPandas sjoin()) on the 
default "intersection" predicate. This allows some tiles 
to contribute to multiple hexagons if they partially overlap more 
than one area.

After computing tile/hexagon intersection, the most recent 4 quarters 
are selected for calculating speed statistics. For many (rural) areas
there are no or few tiles from this; so an expanding 
selection is done to try and get better stats in these areas by 
verifying the sum of tests in the area are at least 10, 
adding first the remainder of the "fixed" data tiles for 
earlier quarter, then all tiles tiles (including mobile tiles)
if that still does not give at least 10 tests. 



In [ ]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

import statsmodels as sm
# import statsmodels.stats.weightstats
from scipy.stats import lognorm

import src.config
from pathlib import Path
import geopandas as gp
import src.datasets.joins as joins

import re

In [ ]:
output_name = "LastFourQuartersOrBestEstimate_On_DissolvedSmallerCitiesHexes"
output_dir = src.config.DATA_DIRECTORY / "processed" / "statistical_geometries"
output_dir.mkdir(exist_ok=True)

In [ ]:
CRS = 'EPSG:4326'

In [ ]:
popctrs = statcan.boundary('population_centres')

In [ ]:
derived_geometry = (Path(src.config.DATA_DIRECTORY) / 'processed' / 'geometries').resolve()

speed_data = gp.read_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs.geojson')#, driver='GeoJSON')
#speed_data = speed_data.to_crs(CRS)
# speed_data.crs = popctrs.crs #assign crs b/c geojson assumes GPS/epsg:4326
speed_data = speed_data.dropna(subset=['HEXUID_PCPUID'])

In [ ]:
speed_data.head(2)

In [ ]:
tiles = ookla.canada_speed_tiles()#.loc[lambda s:(s.year==2022 ) & (s.conn_type=='fixed')]
tiles = tiles.to_crs(CRS)

In [ ]:
unique_tiles = tiles[['quadkey','geometry']].drop_duplicates()
unique_tiles = unique_tiles.sjoin(speed_data[['HEXUID_PCPUID','geometry']].to_crs(tiles.crs))
unique_tiles = unique_tiles.drop(['index_right'],axis=1)

In [ ]:
tiles = tiles.merge(unique_tiles.drop(['geometry'],axis=1), on='quadkey')

In [ ]:
tiles.head(2)

# Questions:
1. How many hex/areas are missing ookla tests/tiles from the last year (or X quarters)?
2. Can we selectively go back to older tiles to fill in areas with no tests in last time period?
3. Can we back fill older tiles/tests in areas with low test counts?

In [ ]:
populated_areas = speed_data.loc[lambda s:s.Pop2016 > 0, 'HEXUID_PCPUID']
populated_areas

In [ ]:
areas = len(populated_areas)
areas

In [ ]:
areas_with_tiles = len(tiles.loc[lambda s:s.HEXUID_PCPUID.isin(populated_areas)].drop_duplicates(subset=['HEXUID_PCPUID']))
areas_with_tiles 

In [ ]:
areas_with_tiles_2022 = len(tiles.loc[lambda s:s.HEXUID_PCPUID.isin(populated_areas)].loc[lambda s:s.year == 2022].drop_duplicates(subset=['HEXUID_PCPUID']))
areas_with_tiles_2022

In [ ]:
areas_with_tiles/len(populated_areas)

In [ ]:
areas_with_tiles_2022/len(populated_areas)

In [ ]:
tiles.loc[lambda s:s.HEXUID_PCPUID.isin(populated_areas)].sort_values(by=['conn_type']).sort_values(by=['year','quarter'], ascending=False).drop_duplicates(subset='HEXUID_PCPUID')[['year','quarter','conn_type']].value_counts(normalize=True)

In [ ]:
# areas where last 4 quarters have at least 10 tests on fixed connections
# take all quarters on fixed tiles with at least 10 tests on fixed connections
# use all tiles

In [ ]:
tests_in_last_4quarters = tiles.loc[lambda s:(s.year==2022) | ( (s.year==2021) & (s.quarter==4) )].loc[lambda s:s.conn_type=='fixed'].groupby('HEXUID_PCPUID')['tests'].sum()
tests_in_last_4quarters

In [ ]:
level_1_areas = tests_in_last_4quarters.index[tests_in_last_4quarters > 10].dropna()
level_1_areas

In [ ]:
fixed_tests_counts_nolevel1 = tiles.loc[lambda s:~s.HEXUID_PCPUID.isin(level_1_areas)].loc[lambda s:s.conn_type == 'fixed'].groupby('HEXUID_PCPUID')['tests'].sum()
fixed_tests_counts_nolevel1

In [ ]:
level_2_areas = fixed_tests_counts_nolevel1.index[fixed_tests_counts_nolevel1 > 10].dropna()
level_2_areas

In [ ]:
level_3_areas = tiles.drop_duplicates(subset='HEXUID_PCPUID').loc[lambda s:~s.HEXUID_PCPUID.isin(level_1_areas) & ~s.HEXUID_PCPUID.isin(level_2_areas)].HEXUID_PCPUID.dropna().values
level_3_areas

In [ ]:
l1_tiles = tiles.loc[lambda s:s.HEXUID_PCPUID.isin(level_1_areas)].loc[lambda s:(s.year==2022) | ( (s.year==2021) & (s.quarter==4) )].loc[lambda s:s.conn_type=='fixed']
l2_tiles = tiles.loc[lambda s:s.HEXUID_PCPUID.isin(level_2_areas)].loc[lambda s:s.conn_type=='fixed']
l3_tiles = tiles.loc[lambda s:s.HEXUID_PCPUID.isin(level_3_areas)]

selected_tiles = pd.concat([l1_tiles, l2_tiles, l3_tiles]).drop_duplicates(subset=['quadkey','conn_type','year','quarter'])
selected_tiles

In [ ]:
speed_data = joins.add_simple_stats(speed_data, selected_tiles, 'HEXUID_PCPUID')

In [ ]:
speed_data[pd.isna(speed_data.num_tiles)]

In [ ]:
speed_data = joins.add_50_10_stats(speed_data, selected_tiles, 'HEXUID_PCPUID')

In [ ]:
unused_columns = ['pc_area','hex_area','pc_frac', 'hex_frac']
for col in unused_columns:
    del speed_data[col]

speed_data['ookla_50_10_percentile'] = speed_data.apply(lambda s:min(s['50_down_percentile'], s['10_up_percentile']), axis=1)

In [ ]:
speed_data.columns

In [ ]:
# speed_data = joins.add_tile_info(speed_data, selected_tiles.drop("HEXUID_PCPUID", axis=1), "HEXUID_PCPUID")
speed_data = joins.add_tile_info(speed_data, selected_tiles, "HEXUID_PCPUID")

In [ ]:
speed_data.columns

In [ ]:
def year_serializeable(tuple_or_na):
    if pd.isna(tuple_or_na):
        return "No Data"
    return "Q{1} {0}".format(*tuple_or_na)

def connections_serializeable(set_):
    if set_ == {'fixed'}:
        return 'fixed'
    elif set_ == {'fixed','mobile'}:
        return 'fixed and mobile'
    elif set_ == {'mobile'}:
        return 'mobile'
    else:
        return 'No Data'


pat = '(\d\d)p_*'
t = '25p_d_kbps'
fix_bad_names = lambda x: re.sub(pat, lambda s:f'P{s.group(1)}_', x)

In [ ]:
speed_data['min_year'] = speed_data['min_year'].apply(year_serializeable)
speed_data['max_year'] = speed_data['max_year'].apply(year_serializeable)
speed_data['connections'] = speed_data['connections'].apply(connections_serializeable)

speed_data.rename(columns=fix_bad_names, inplace=True)
speed_data.rename(columns={'50_down_percentile':'Down_50_percentile', '10_up_percentile':'Up_10_percentile'}, inplace=True)

In [ ]:
speed_data.rename(columns=fix_bad_names, inplace=True)
speed_data.rename(columns={'50_down_percentile':'Down_50_percentile', '10_up_percentile':'Up_10_percentile'}, inplace=True)

In [ ]:
speed_data.crs

In [ ]:
tiles.loc[lambda s:s.HEXUID_PCPUID == "AB56801115"].sort_values(by=["year","quarter"],ascending=False)

In [ ]:
speed_data.loc[lambda s:s.PRCODE == "AB"].loc[lambda s:s.Pop2016>0].explore(
    'ookla_50_10_percentile',scheme='equalinterval', k = 4, 
    tooltip=['HEXUID_PCPUID','PCNAME','Pop2016','Pop_Avail_50_10','ookla_50_10_percentile'],
    popup=[
        'HEXUID_PCPUID','PCNAME',
        'min_d_kbps','avg_d_kbps','max_d_kbps',
        'min_u_kbps','avg_u_kbps','max_u_kbps',
        'Pop2016','tests','num_tiles','unique_devices', 'min_year','max_year','connections',
        'Pop_Avail_50_10','ookla_50_10_percentile','Down_50_percentile','Up_10_percentile']
    )

In [ ]:
"AB56801115"

In [ ]:
speed_data.loc[lambda s:(s.Pop2016 > 0.0) | (s.tests > 0.0)].to_file(output_dir / output_name, driver="MapInfo File")


In [ ]:
speed_data.loc[lambda s:(s.Pop2016 > 0.0) | (s.tests > 0.0)].to_file(output_dir / (output_name+".gpkg"), driver="GPKG")

In [ ]:
speed_data.columns

In [ ]:
output_dir / (output_name+"gpkg")

In [ ]:
# reloaded_speed_data = gp.read_file(output_dir / output_name, driver="MapInfo File") ## doesn't encode nans; converts to zeros T_T
reloaded_speed_data = gp.read_file(output_dir / (output_name+".gpkg"), driver="GPKG")

In [ ]:
reloaded_speed_data.loc[lambda s:s.PCNAME=='Brooks']

In [ ]:
reloaded_speed_data.loc[lambda s:s.Pop2016 >0.0].loc[lambda s:s.PRCODE=="AB"].explore(
    'ookla_50_10_percentile',scheme='equalinterval', k = 4, 
    tooltip=['HEXUID_PCPUID','PCNAME','Pop2016','Pop_Avail_50_10','ookla_50_10_percentile'],
    popup=[
        'HEXUID_PCPUID','PCNAME',
        'min_d_kbps','avg_d_kbps','max_d_kbps',
        'min_u_kbps','avg_u_kbps','max_u_kbps',
        'Pop2016','tests','num_tiles','unique_devices', 'min_year','max_year','connections',
        'Pop_Avail_50_10','ookla_50_10_percentile','Down_50_percentile','Up_10_percentile']
    )

In [ ]:
speed_data.columns

In [ ]:
speed_data.plot.scatter(x='avg_d_kbps',y='P50_d_kbps',alpha=0.2)

In [ ]:
speed_data['avg_d_kbps'].plot.hist(bins=range(0,100000,1000))
plt.gca().axvline(50000,color='k')

In [ ]:
speed_data.loc[lambda s:~(s.PCCLASS == '4')]['P75_d_kbps'].plot.hist(bins=range(0,400000,5000), density=True, alpha=0.75)
speed_data.loc[lambda s:(s.PCCLASS == '4')]['P75_d_kbps'].plot.hist(bins=range(0,400000,5000), density=True,alpha=0.75)
plt.gca().axvline(50000,color='k')

In [ ]:
speed_data.loc[lambda s:pd.isna(s.PCCLASS)]['P75_u_kbps'].plot.hist(bins=range(0,250000,5000), density=True, alpha=0.75)
speed_data.loc[lambda s:~pd.isna(s.PCCLASS)]['P75_u_kbps'].plot.hist(bins=range(0,250000,5000), density=True, alpha=0.75)
plt.gca().axvline(10000,color='k')

In [ ]:
speed_data.loc[lambda s:pd.isna(s.PCPUID)]['P75_u_kbps'].plot.hist(bins=range(0,200000,5000))
plt.gca().axvline(10000,color='k')

In [ ]:
ax = speed_data.loc[lambda s:(s.Pop_Avail_50_10 > 0) & (s.ookla_50_10_percentile)].loc[lambda s:s.PCCLASS.isna()].plot.scatter(x='Pop_Avail_50_10', y='ookla_50_10_percentile', alpha=.1)
ax.plot([0,100], [0,100], color='k', lw=2);

In [ ]:
thing =speed_data.loc[lambda s:(s.Pop_Avail_50_10 > 0) & (s.ookla_50_10_percentile)]
plt.hist2d(thing['Pop_Avail_50_10'], thing['ookla_50_10_percentile'], bins=20, range=[(0,100),(0,100)]);

In [ ]:
speed_data.loc[lambda s:s.Pop_Avail_50_10>0].apply(lambda s:s.ookla_50_10_percentile/s.Pop_Avail_50_10 if s.Pop_Avail_50_10 > 0 else 0, axis=1).hist(bins=20, range=[0,2])

In [ ]:
speed_data.loc[lambda s:s.Pop_Avail_50_10>0].apply(lambda s:s.ookla_50_10_percentile-s.Pop_Avail_50_10, axis=1).hist(bins=50)#, range=[0,2])


In [ ]:
speed_data

In [ ]:
speed_data.Pop2016.sum()

In [ ]:
print("{:10,.0f} >50/10;\n{:10,.0f} <50/10;\n{:10,.0f} no data;\n{:10,.0f} all pop".format(
*[
    (speed_data['Pop2016']*speed_data['ookla_50_10_percentile']/100.0).sum(),
    (speed_data['Pop2016']*(100-speed_data['ookla_50_10_percentile'])/100.0).sum(),
    speed_data[speed_data['ookla_50_10_percentile'].isna()].Pop2016.sum(),
    speed_data.Pop2016.sum()
]
)
)

In [ ]:
28_820_499/35_151_728*100

In [ ]:
print("{:10,.0f} >50/10;\n{:10,.0f} <50/10;\n{:10,.0f} no data;\n{:10,.0f} all pop".format(
*[
    (rural_speed_data['Pop2016']*rural_speed_data['ookla_50_10_percentile']/100.0).sum(),
    (rural_speed_data['Pop2016']*(100-rural_speed_data['ookla_50_10_percentile'])/100.0).sum(),
    rural_speed_data[rural_speed_data['ookla_50_10_percentile'].isna()].Pop2016.sum(),
    rural_speed_data.Pop2016.sum(),
]
)
)

In [ ]:
2_963_931/6_577_998*100

In [ ]:
speed_data.columns

In [ ]:
print("{:10,.0f} >50/10;\n{:10,.0f} <50/10;\n{:10,.0f} no data;\n{:10,.0f} all pop".format(
*[
    (speed_data['Pop2016']*speed_data['Pop_Avail_50_10']/100).sum(),
    (speed_data['Pop2016']*(100-speed_data['Pop_Avail_50_10'])/100).sum(),
    speed_data[speed_data['Pop_Avail_50_10'].isna()].Pop2016.sum(),
    speed_data.Pop2016.sum()
]
)
)

In [ ]:
32458702/35151728*100

In [ ]:
print("{:10,.0f} >50/10;\n{:10,.0f} <50/10;\n{:10,.0f} no data;\n{:10,.0f} all pop".format(
*[
    (rural_speed_data['Pop2016']*rural_speed_data['Pop_Avail_50_10']/100.0).sum(),
    (rural_speed_data['Pop2016']*(100-rural_speed_data['Pop_Avail_50_10'])/100.0).sum(),
    rural_speed_data[rural_speed_data['Pop_Avail_50_10'].isna()].Pop2016.sum(),
    rural_speed_data.Pop2016.sum(),
]
)
)

In [ ]:
4_108_512/6_577_998*100